In [2]:
pip install pygame numpy


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pygame
import sys
import numpy as np
import copy
import random

pygame.init()

# --- Constants ---
WIDTH, HEIGHT = 320, 380
BOARD_ROWS, BOARD_COLS = 3, 3
SQUARE_SIZE = WIDTH // BOARD_COLS
LINE_WIDTH = 5
CIRCLE_RADIUS = SQUARE_SIZE // 3
CIRCLE_WIDTH = 12
CROSS_WIDTH = 18
SPACE = SQUARE_SIZE // 4

# --- Colors ---
BG_COLOR = (30, 30, 30)
LINE_COLOR = (200, 200, 200)
CIRCLE_COLOR = (70, 130, 180)
CROSS_COLOR = (255, 140, 0)
WIN_LINE_COLOR = (50, 205, 50)
TEXT_COLOR = (240, 240, 240)

# --- Setup Screen ---
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Tic Tac Toe')
screen.fill(BG_COLOR)
font = pygame.font.SysFont("Arial", 24)

# --- Game Variables ---
board = np.zeros((BOARD_ROWS, BOARD_COLS), dtype=int).tolist()
player = 2  # AI starts
game_over = False
current_winner_player = None

# --- Drawing Functions ---
def draw_title():
    title_surface = font.render("Tic Tac Toe (Press R to Restart)", True, TEXT_COLOR)
    screen.blit(title_surface, (20, HEIGHT - 40))

def draw_lines():
    for i in range(1, BOARD_ROWS):
        pygame.draw.line(screen, LINE_COLOR, (0, SQUARE_SIZE * i), (WIDTH, SQUARE_SIZE * i), LINE_WIDTH)
    for i in range(1, BOARD_COLS):
        pygame.draw.line(screen, LINE_COLOR, (SQUARE_SIZE * i, 0), (SQUARE_SIZE * i, WIDTH), LINE_WIDTH)

def draw_figures():
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            center = (col * SQUARE_SIZE + SQUARE_SIZE // 2, row * SQUARE_SIZE + SQUARE_SIZE // 2)
            if board[row][col] == 1:
                pygame.draw.circle(screen, CIRCLE_COLOR, center, CIRCLE_RADIUS, CIRCLE_WIDTH)
            elif board[row][col] == 2:
                pygame.draw.line(screen, CROSS_COLOR,
                                 (col * SQUARE_SIZE + SPACE, row * SQUARE_SIZE + SPACE),
                                 (col * SQUARE_SIZE + SQUARE_SIZE - SPACE, row * SQUARE_SIZE + SQUARE_SIZE - SPACE),
                                 CROSS_WIDTH)
                pygame.draw.line(screen, CROSS_COLOR,
                                 (col * SQUARE_SIZE + SPACE, row * SQUARE_SIZE + SQUARE_SIZE - SPACE),
                                 (col * SQUARE_SIZE + SQUARE_SIZE - SPACE, row * SQUARE_SIZE + SPACE),
                                 CROSS_WIDTH)

def draw_win_line(row=None, col=None, diagonal=None):
    if diagonal == "desc":
        pygame.draw.line(screen, WIN_LINE_COLOR, (20, 20), (WIDTH - 20, WIDTH - 20), LINE_WIDTH)
    elif diagonal == "asc":
        pygame.draw.line(screen, WIN_LINE_COLOR, (20, WIDTH - 20), (WIDTH - 20, 20), LINE_WIDTH)
    elif row is not None:
        y = row * SQUARE_SIZE + SQUARE_SIZE // 2
        pygame.draw.line(screen, WIN_LINE_COLOR, (20, y), (WIDTH - 20, y), LINE_WIDTH)
    elif col is not None:
        x = col * SQUARE_SIZE + SQUARE_SIZE // 2
        pygame.draw.line(screen, WIN_LINE_COLOR, (x, 20), (x, WIDTH - 20), LINE_WIDTH)

# --- Game Logic Functions (unchanged) ---
def mark_square(row, col, player_num):
    board[row][col] = player_num

def available_square(row, col):
    return board[row][col] == 0

def is_board_full(check_board=None):
    check_board = check_board or board
    return all(cell != 0 for row in check_board for cell in row)

def check_win(player_num, check_board=None):
    check_board = check_board or board
    for col in range(BOARD_COLS):
        if all(check_board[row][col] == player_num for row in range(BOARD_ROWS)):
            return "col", col
    for row in range(BOARD_ROWS):
        if all(check_board[row][col] == player_num for col in range(BOARD_COLS)):
            return "row", row
    if all(check_board[i][i] == player_num for i in range(BOARD_ROWS)):
        return "desc", None
    if all(check_board[i][BOARD_COLS - 1 - i] == player_num for i in range(BOARD_ROWS)):
        return "asc", None
    return None, None

def minimax(minimax_board, depth, is_maximizing):
    if check_win(2, minimax_board)[0]:
        return 1
    elif check_win(1, minimax_board)[0]:
        return -1
    elif is_board_full(minimax_board):
        return 0

    if is_maximizing:
        best_score = -float('inf')
        for row in range(BOARD_ROWS):
            for col in range(BOARD_COLS):
                if minimax_board[row][col] == 0:
                    minimax_board[row][col] = 2
                    score = minimax(minimax_board, depth + 1, False)
                    minimax_board[row][col] = 0
                    best_score = max(score, best_score)
        return best_score
    else:
        best_score = float('inf')
        for row in range(BOARD_ROWS):
            for col in range(BOARD_COLS):
                if minimax_board[row][col] == 0:
                    minimax_board[row][col] = 1
                    score = minimax(minimax_board, depth + 1, True)
                    minimax_board[row][col] = 0
                    best_score = min(score, best_score)
        return best_score

def best_move():
    global board
    move = None
    use_minimax = random.random() < 0.3  # Only 30% chance to use minimax

    if use_minimax:
        best_score = -float('inf')
        for row in range(BOARD_ROWS):
            for col in range(BOARD_COLS):
                if board[row][col] == 0:
                    temp_board = copy.deepcopy(board)
                    temp_board[row][col] = 2
                    score = minimax(temp_board, 0, False)
                    if score > best_score:
                        best_score = score
                        move = (row, col)
    else:
        empty = [(r, c) for r in range(BOARD_ROWS) for c in range(BOARD_COLS) if board[r][c] == 0]
        if empty:
            move = random.choice(empty)

    if move:
        mark_square(move[0], move[1], 2)
        return True
    return False


def restart_game():
    global board, player, game_over, current_winner_player
    board = np.zeros((BOARD_ROWS, BOARD_COLS), dtype=int).tolist()
    player = 2
    game_over = False
    current_winner_player = None
    screen.fill(BG_COLOR)
    draw_lines()
    best_move()  # AI first
    draw_figures()
    draw_title()
    player = 1


draw_lines()
best_move()
draw_figures()
draw_title()
player = 1


while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        if event.type == pygame.MOUSEBUTTONDOWN and not game_over and player == 1:
            mouseX, mouseY = event.pos
            if mouseY < WIDTH:
                clicked_col = mouseX // SQUARE_SIZE
                clicked_row = mouseY // SQUARE_SIZE

                if available_square(clicked_row, clicked_col):
                    mark_square(clicked_row, clicked_col, 1)
                    result = check_win(1)
                    if result[0]:
                        game_over = True
                        current_winner_player = 1
                    else:
                        player = 2
                    screen.fill(BG_COLOR)
                    draw_lines()
                    draw_figures()
                    draw_title()

        if event.type == pygame.KEYDOWN and event.key == pygame.K_r:
            restart_game()

    if player == 2 and not game_over:
        if best_move():
            result = check_win(2)
            if result[0]:
                game_over = True
                current_winner_player = 2
            elif is_board_full(board):
                game_over = True
            else:
                player = 1
        screen.fill(BG_COLOR)
        draw_lines()
        draw_figures()
        draw_title()

    if game_over and current_winner_player:
        result_type, index = check_win(current_winner_player)
        if result_type == "row":
            draw_win_line(row=index)
        elif result_type == "col":
            draw_win_line(col=index)
        elif result_type in ["asc", "desc"]:
            draw_win_line(diagonal=result_type)

    pygame.display.update()
